## **Librerias**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## **URL**

In [2]:

    
# URL de la sección de componentes de PC
section_url = "https://listado.mercadolibre.com.ar/componentes-de-pc#D[A:Componentes%20de%20PC]" 

# Encabezados para evitar bloqueos
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-ES,es;q=0.9'
}


## **Funciones**

In [3]:
# Función para obtener enlaces de productos desde múltiples páginas de una misma seccion
def get_product_links(base_url, num_pages=5):
    product_links = []
    for i in range(num_pages):
        # Construir la URL para cada página
        page_offset = i * 48
        url = f"{base_url}_Desde_{page_offset}_NoIndex_True" if page_offset > 0 else base_url

        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar todos los enlaces de productos en la página actual
        for a_tag in soup.find_all("a", href=True):
            href = a_tag["href"]
            # Filtrar enlaces que parecen productos
            if "mercadolibre.com.ar" in href and "/p/" in href:
                full_link = href.split("?")[0]  # Limpiar parámetros adicionales
                if full_link not in product_links:  # Evitar duplicados
                    product_links.append(full_link)

    return product_links

# Función para clasificar el tipo de producto
def classify_product_type(title):
    title_lower = title.lower()
    if "procesador" in title_lower:
        return "Procesador"
    elif "ram" in title_lower:
        return "RAM"
    elif "placa de video" in title_lower or "tarjeta de video" in title_lower or "placa video" in title_lower or "tarjeta video" in title_lower:
        return "Placa de Video"
    elif "disco" in title_lower or "ssd" in title_lower or "hdd" in title_lower:
        return "Disco"
    elif "gabinete" in title_lower:
        return "Gabinete"
    elif "placa madre" in title_lower or "tarjeta madre" in title_lower or "motherboard" in title_lower:
        return "Placa Madre"
    elif "computadora" in title_lower or "pc" in title_lower:
        return "Computadora"
    elif "cooler" in title_lower:
        return "Cooler"
    elif "térmica" in title_lower or "pasta térmica" in title_lower:
        return "Pasta Térmica"
    else:
        return "Otro"

# Función para extraer información de un producto desde la pagina de venta del producto
def extract_product_data(URL):
    # Realizar la solicitud a la página del producto
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Inicializar los datos del producto
    product_data = {}

    # Título del producto
    try:
        title = soup.find("h1", class_="ui-pdp-title").text.strip()
        product_data['Title'] = title
        # Clasificar el tipo de producto
        product_data['Type'] = classify_product_type(title)
    except AttributeError:
        product_data['Title'] = "NA"
        product_data['Type'] = "NA"

    # Precio
    try:
        price = soup.find("span", class_="andes-money-amount__fraction").text.strip().replace(".", "")
        product_data['Price'] = int(price)  # Convertir a entero
    except AttributeError:
        product_data['Price'] = "NA"

    # Calificación
    try:
        rating = soup.find("span", class_="ui-pdp-review__rating").text.strip().replace(".", ",")
        product_data['Rating'] = rating  # Mantener como texto con coma
    except AttributeError:
        product_data['Rating'] = "NA"

   # Número de opiniones
    try:
        reviews = soup.find("span", class_="ui-pdp-review__amount").text.strip("()").replace(".", "")
        product_data['Reviews'] = int(reviews)  # Convertir a entero
    except AttributeError:
        product_data['Reviews'] = "NA"

   # Cantidad de productos vendidos
    try:
        sold_text = soup.find("span", class_="ui-pdp-subtitle").text.strip()
        if "vendidos" in sold_text:
            sold_number = (
                sold_text.split("|")[-1]
                .replace("vendidos", "")
                .replace("+", "")
                .replace("mil", "000")
                .strip()
            )
            product_data['Sold'] = int(sold_number)
        else:
            product_data['Sold'] = "NA"
    except AttributeError:
        product_data['Sold'] = "NA"

    return product_data



## **Main**

In [4]:
# Main
if __name__ == '__main__':
    # Obtener enlaces de productos de las primeras 5 páginas
    num_pages = 10  # Cambia este valor para ajustar el número de páginas
    product_links = get_product_links(section_url, num_pages=num_pages)
    print(f"Se encontraron {len(product_links)} enlaces únicos.")
    # Crear una lista para almacenar los datos de los productos
    all_products = []
    # Iterar sobre los enlaces y extraer datos
    for link in product_links:
        product_data = extract_product_data(link)
        all_products.append(product_data)

    # Crear un DataFrame y guardar como CSV
    df = pd.DataFrame(all_products)
    df.to_csv("productos_mercadolibre.csv", index=False, encoding='utf-8-sig')
    print("Datos guardados en 'productos_mercadolibre.csv'.")


Se encontraron 479 enlaces únicos.
Datos guardados en 'productos_mercadolibre.csv'.
